# Materials Project Data Extraction

In [1]:
from mp_api.client import MPRester
API_KEY ='YUgbwowwn50V8bL2u8H3u8mvIVchWLpL'

No module named 'phonopy'
No module named 'phonopy'


In [2]:

material_ids = []
material_docs = []
with MPRester(API_KEY) as mpr:
    docs =  mpr.materials.summary.search(elements=["C", "S"], 
                              num_elements =(2,2))
    for doc in docs:
        material_docs.append(doc)
        material_ids.append(doc.material_id)
        

Retrieving SummaryDoc documents:   0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.sets import Poscar
for material_id in material_ids:
    with MPRester(API_KEY) as mpr:
        structure = mpr.materials.get_structure_by_material_id(material_id)
        # 生成POSCAR对象
        poscar = Poscar(structure)
        # 将POSCAR对象写入文件
        poscar.write_file("poscar/POSCAR_{}.vasp".format(material_id))
        

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
import pandas as pd

# Assuming material_docs and material_ids are already defined and populated with your data
data_rows = []

# Validate that we have materials to process and that material_ids are provided
if material_docs and material_ids and len(material_docs) == len(material_ids):
    # Generate column names from the attributes of the first document, excluding 'material_id'
    columns = [attr for attr in dir(material_docs[0]) if not attr.startswith('__') and not callable(getattr(material_docs[0], attr))]
    
    # Construct data rows by combining material_ids with other attributes of documents
    for material_id, doc in zip(material_ids, material_docs):
        row = [material_id] + [getattr(doc, col) for col in columns if hasattr(doc, col)]
        data_rows.append(row)
    
    # Create DataFrame with 'material_id' as the first column
    df = pd.DataFrame(data_rows, columns=['material_id'] + columns)

    filename = 'materials_information.xlsx'
    df.to_excel(filename, index=False)

    print(f'Data successfully written to {filename}')
else:
    print("Check that material_docs and material_ids are properly defined and have matching lengths.")


Data successfully written to materials_information.xlsx


In [18]:
from server.app.pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient(host='0.0.0.0', port=27017)  # Adjust host and port as necessary
db = client['materialdb']  # Database name
collection = db['materials_collection']  # Collection name

# Load Excel file
filename = 'materials_information.xlsx'  # Update this to the path of your Excel file
df = pd.read_excel(filename)

# Convert DataFrame to dictionary and insert into MongoDB
documents = df.to_dict(orient='records')  # Converts each row to a dictionary with column names as keys

# Insert documents into MongoDB
result = collection.insert_many(documents)

# Print the result of the insertion
print(f"Inserted {len(result.inserted_ids)} documents into MongoDB.")


Inserted 12 documents into MongoDB.


In [23]:
def convert_to_poscar(input_text):
    lines = input_text.strip().split('\n')
    try:
        scaling_factor = float(lines[1].strip())
        lattice_vectors = lines[2:5]
        species_line = lines[6]
        count_line = lines[7]
        positions = lines[9:]

        # Process lattice vectors
        formatted_lattice_vectors = [
            f"{float(vec.split()[0]):>20.10f} {float(vec.split()[1]):>20.10f} {float(vec.split()[2]):>20.10f}"
            for vec in lattice_vectors
        ]

        # Process species and counts
        species = species_line.strip().split()
        counts = count_line.strip().split()
        species_count = zip(species, counts)

        # Format header information
        poscar_output = [
            "POSCAR(15)",
            f"{scaling_factor:.1f}",
            *formatted_lattice_vectors,
            "   " + "    ".join(species),
            "   " + "    ".join(counts),
            "Direct"
        ]

        # Process and format positions
        formatted_positions = [
            f"{float(pos.split()[0]):>16.10f} {float(pos.split()[1]):>16.10f} {float(pos.split()[2]):>16.10f} {pos.split()[3]}"
            for pos in positions if pos.strip()  # Ensure there's data on the line
        ]
        poscar_output.extend(formatted_positions)

        return "\n".join(poscar_output)

    except IndexError as e:
        return f"Error processing input: {str(e)}"
    except ValueError as e:
        return f"Error processing numbers: {str(e)}"


In [25]:
old ="""C1 S30
1.0
  18.5068620000000017    7.8089940000000002    0.0000000000000000
 -18.5068620000000017    7.8089940000000002    0.0000000000000000
   0.0000000000000000    0.6499130000000000   16.4418429999999987
C S
1 30
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C
   0.5257849999999999    0.4744880000000000    0.2660380000000000 S
   0.4742150000000000    0.5255120000000000    0.7339620000000000 S
   0.4607620000000000    0.2627350000000000    0.5372990000000000 S
   0.5255120000000000    0.4742150000000000    0.7339620000000000 S
   0.5392380000000000    0.7372649999999999    0.4627010000000000 S
   0.4744880000000000    0.5257849999999999    0.2660380000000000 S
   0.2546660000000000    0.5261160000000000    0.4598930000000000 S
   0.7372649999999999    0.5392380000000000    0.4627010000000000 S
   0.7453340000000001    0.4738840000000000    0.5401070000000000 S
   0.2627350000000000    0.4607620000000000    0.5372990000000000 S
   0.4738840000000000    0.7453340000000001    0.5401070000000000 S
   0.5261160000000000    0.2546660000000000    0.4598930000000000 S
   0.0049790000000000    0.3169380000000000    0.4996920000000000 S
   0.9950210000000000    0.6830619999999999    0.5003080000000000 S
   0.3426900000000000    0.5352830000000000    0.0000910000000000 S
   0.6830619999999999    0.9950210000000000    0.5003080000000000 S
   0.6573099999999999    0.4647170000000000    0.9999090000000000 S
   0.3169380000000000    0.0049790000000000    0.4996920000000000 S
   0.4997530000000000    0.0000840000000000    0.3134800000000000 S
   0.4647170000000000    0.6573099999999999    0.9999090000000000 S
   0.5002470000000000    0.9999160000000000    0.6865200000000000 S
   0.5352830000000000    0.3426900000000000    0.0000910000000000 S
   0.9999160000000000    0.5002470000000000    0.6865200000000000 S
   0.0000840000000000    0.4997530000000000    0.3134800000000000 S
   0.9977680000000000    0.9977680000000000    0.0944910000000000 S
   0.0022320000000000    0.0022320000000000    0.9055090000000000 S
   0.9678220000000000    0.1538000000000000    0.9985140000000000 S
   0.0321780000000000    0.8462000000000000    0.0014860000000000 S
   0.1538000000000000    0.9678220000000000    0.9985140000000000 S
   0.8462000000000000    0.0321780000000000    0.0014860000000000 S
"""

new = convert_to_poscar(old)
# write new into a file
with open('new_poscar.vasp', 'w') as f:
    f.write(new)

'POSCAR(15)\n1.0\n       18.5068620000         7.8089940000         0.0000000000\n      -18.5068620000         7.8089940000         0.0000000000\n        0.0000000000         0.6499130000        16.4418430000\n   1    30\n   direct\nDirect\n    0.5257850000     0.4744880000     0.2660380000 S\n    0.4742150000     0.5255120000     0.7339620000 S\n    0.4607620000     0.2627350000     0.5372990000 S\n    0.5255120000     0.4742150000     0.7339620000 S\n    0.5392380000     0.7372650000     0.4627010000 S\n    0.4744880000     0.5257850000     0.2660380000 S\n    0.2546660000     0.5261160000     0.4598930000 S\n    0.7372650000     0.5392380000     0.4627010000 S\n    0.7453340000     0.4738840000     0.5401070000 S\n    0.2627350000     0.4607620000     0.5372990000 S\n    0.4738840000     0.7453340000     0.5401070000 S\n    0.5261160000     0.2546660000     0.4598930000 S\n    0.0049790000     0.3169380000     0.4996920000 S\n    0.9950210000     0.6830620000     0.5003080000 S\n   

In [ ]:
list_of_available_fields